In [19]:
from tensorflow.keras.models import Sequential # НС прямого распространения
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization # Основные слои
from tensorflow.keras import utils # Утилиты для to_categorical
from tensorflow.keras.preprocessing import image # Для отрисовки изображения
from tensorflow.keras.optimizers import Adam, Adadelta # Алгоритмы оптимизации, для настройки скорости обучения
import numpy as np # Библиотека работы с массивами
import matplotlib.pyplot as plt # Отрисовка изображений
from PIL import Image # Отрисовка изображений
import pandas as pd # Библиотека pandas
# from google.colab import files # Импорт файлов
from sklearn.preprocessing import LabelEncoder, StandardScaler # Функции для нормализации данных
from sklearn import preprocessing # Пакет предварительной обработки данных
from sklearn.model_selection import train_test_split
import category_encoders as ce
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.compose import ColumnTransformer

In [3]:
cars = pd.read_csv('cars_new.csv', sep=',')

In [5]:
X = cars.drop('price', axis=1)
y = cars['price']

In [13]:
num_cols = X.dtypes[X.dtypes != 'object'].keys()
cat_cols = X.dtypes[X.dtypes == 'object'].keys()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [21]:
pl_transform = Pipeline([
            ('ohe', ce.OneHotEncoder(cols=cat_cols)),
            # ('scaler', ColumnTransformer([('ct', RobustScaler(), ['MonthlySpending', 'TotalSpent'])],remainder='passthrough')),
            ('scaler',  StandardScaler()),
        ])
X_train_transf = pl_transform.fit_transform(X_train)
X_test_transf = pl_transform.transform(X_test)

C:\Users\Alex\anaconda3\lib\site-packages\category_encoders\utils.py:21: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  elif pd.api.types.is_categorical(cols):


In [16]:
# Нормализуем y_train
y_scaler = StandardScaler()
y_train_scaled = y_scaler.fit_transform(y_train.reshape(-1,1)).flatten()

In [ ]:
model_ula = Sequential()
model_ula.add(Dense(800, input_dim=X_train_transf.shape[1], activation='relu'))
model_ula.add(Dropout(0.2))
model_ula.add(Dense(400, activation='relu'))
model_ula.add(Dropout(0.2))
model_ula.add(Dense(1))

model_ula.compile(optimizer=Adam(), loss='mse', metrics='mae')
model_ula.fit(X_train_transf, y_train_scaled, batch_size=128, epochs=10)
# training
predict = model_ula.predict(X_test_transf)